# Inequalities testing

Этот код нужен, чтобы проверять придуманный сертификат на адекватность - введи ограничения на углы и задай вектор u в функции func(var) и код попробует найти углы, на которых знаки Au будут разные.

In [8]:
import sympy
import mystic.symbolic as ms
import numpy as np
from numpy import cos, sin
import re
import operator
import pprint

In [3]:
var = list("ABCDEFG")
add_eqs = []

for _ in range(len(var)):
    s = ' + '.join(var[:4]) + ' - ' + ' - '.join(var[4:])
    add_eqs.append(s + ' < 3.141592653589793')
    add_eqs.append(s + ' > -3.141592653589793')
    var.append(var.pop(0))

add_eqs.append('A + B + C + D + E + F + G > 3.141592653589793')
add_eqs.append('A + B + C + D + E + F + G < 9.42477796076938')

add_eqs = '\n'.join(add_eqs)

In [4]:
pprint.pprint(add_eqs)

('A + B + C + D - E - F - G < 3.141592653589793\n'
 'A + B + C + D - E - F - G > -3.141592653589793\n'
 'B + C + D + E - F - G - A < 3.141592653589793\n'
 'B + C + D + E - F - G - A > -3.141592653589793\n'
 'C + D + E + F - G - A - B < 3.141592653589793\n'
 'C + D + E + F - G - A - B > -3.141592653589793\n'
 'D + E + F + G - A - B - C < 3.141592653589793\n'
 'D + E + F + G - A - B - C > -3.141592653589793\n'
 'E + F + G + A - B - C - D < 3.141592653589793\n'
 'E + F + G + A - B - C - D > -3.141592653589793\n'
 'F + G + A + B - C - D - E < 3.141592653589793\n'
 'F + G + A + B - C - D - E > -3.141592653589793\n'
 'G + A + B + C - D - E - F < 3.141592653589793\n'
 'G + A + B + C - D - E - F > -3.141592653589793\n'
 'A + B + C + D + E + F + G > 3.141592653589793\n'
 'A + B + C + D + E + F + G < 9.42477796076938')


In [5]:
def check_linear_constraints(var: dict, equations: str, verbose=False):
    
    comps = ['<', '>', '<=', '>=']
    COMPARES = {'<': operator.lt, '>': operator.gt, '<=': operator.le, '>=': operator.ge}
    OPERATIONS = {'+': operator.add, '-': operator.sub}
    RES = True
    
    for eq in equations.split('\n'):
        
        if not len(eq):
            continue
                
        # Choose a comparison that has to be done
        COMP = ''
        for comp in comps:
            if comp in eq:
                COMP = comp
        
        # Calculate the value on the left
        left, right = re.split('<=|>=|<|>',eq); left.strip(); right.strip()
        right_val = float(right)
        
        val = 0; op = '+'
        
        for el in left.split(' '):
            
            # Add an element
            if el in var.keys():
                #print(el, var[el])
                val = OPERATIONS[op](val, var[el])
            # Change operation
            elif el in OPERATIONS.keys():
                #print(op)
                op = el
            #print(val)
        # Make a comparison
        if verbose:
            if not COMPARES[COMP](val, right_val):
                print(eq)
                print(val, right_val)
                print(COMPARES[COMP](val, right_val))
        RES = RES and COMPARES[COMP](val, right_val)
    if verbose:
        print("Final: ", RES)
    
    return RES

In [6]:
equations = """A + B < 3.141592653589793
A + B > 0
B + C < 3.141592653589793
B + C > 0
C + D < 3.141592653589793
C + D > 0
D + E < 3.141592653589793
D + E > 0
E + F < 3.141592653589793
E + F > 0
F + G < 3.141592653589793
F + G > 0
"""
equations += "\n"+add_eqs

In [9]:
eqns = ms.simplify(equations, variables=var, join="and_") + '\n'+ """A > 0
B > 0
C > 0
D > 0
E > 0
F > 0
G > 0"""

In [10]:
constrain = ms.generate_constraint(ms.generate_solvers(eqns, var))

In [ ]:
def func(var: dict, u):
    
    A = np.array([
        [1, cos(var['A'] + var['B']), -cos(var['E']+var['F']+var['G']), cos(var['F']+var['G'])],
        [cos(var['A'] + var['B']), 1, cos(var['C'] + var['D']), -cos(var['C'] + var['D'] + var['E'])],
        [-cos(var['E']+var['F']+var['G']), cos(var['C'] + var['D']), 1, -cos(var['E'])],
        [cos(var['F']+var['G']), -cos(var['C'] + var['D'] + var['E']), -cos(var['E']), 1]
    ])
        
    return A@u

In [ ]:
def count_charaacteristics(var: dict, angles: list):
    
    angles = angles.copy()
    
    print("Суммы пар:")
    for i in range(len(angles)-1):
        print(var[angles[i]] + var[angles[i+1]])
    
    print("Сумма:")
    print(sum(var.values()))
    
    print("Сигмы:")
    for _ in range(len(var)):
        print(sum([var[x] for x in angles[:4]]) - sum([var[x] for x in angles[4:]]))
        angles.append(angles.pop(0))

In [ ]:
MAX_ITER = 10000

for _ in range(MAX_ITER):
    a = np.random.uniform(0, np.pi, 7)
    solution = constrain(a)
    sol = {k: v for k,v in zip(var, solution)}
    if check_linear_constraints(sol, equations, verbose=False):
        
        certs = [
            np.array([1,0,0,0]),
            np.array([0,1,0,0]),
            np.array([0,0,1,0]),
            np.array([0,0,0,1]),
            np.array([1,1,0,0]),
            np.array([1,0,1,0]),
            np.array([1,0,0,1]),
            np.array([0,1,1,0]),
            np.array([0,1,0,1]),
            np.array([0,0,1,1]),
            np.array([1,1,1,1]),
            np.array([1,1,1,0]),
            np.array([1,1,0,1]),
            np.array([1,0,1,1]),
            np.array([0,1,1,1])
        ]
        
        s = map(lambda x: sum(func(sol, x) >= 0), certs)
        res = map(lambda x: x == 0 or x == len(func(sol, certs[0])), s) 
        # Check that all signs are different
        if not any(res):
            #pprint.pprint(sol)
            #print(list(map(lambda x: func(sol,x), certs)))
            print("____________________")
            count_charaacteristics(sol, var)

## Just generate some points

In [14]:
from pprint import pprint

In [17]:
MAX_ITER = 10

for _ in range(MAX_ITER):
    #a = np.random.uniform(0.1, np.pi-0.1, 7)
    a = np.random.uniform(0, np.pi, 7)
    solution = constrain(a)
    pprint(solution)

array([1.59317582, 0.43959732, 0.45791466, 1.32246291, 1.81912974,
       0.60823814, 0.84957669])
array([0.59925339, 1.32309979, 1.81849287, 0.44503116, 1.51425836,
       0.89589361, 0.73844345])
array([1.58401055, 1.37131369, 0.18626841, 2.52319093, 0.61840173,
       1.57698146, 1.56461119])
array([2.40470309, 0.61696029, 1.32101377, 1.82057888, 1.05863359,
       0.8084145 , 1.39447384])
array([0.27863145, 0.48424409, 0.99983325, 2.1417594 , 0.24891867,
       0.17828762, 1.90322754])
array([1.61936581, 1.50528048, 1.63631218, 0.94832561, 2.19326704,
       0.3661111 , 1.15611574])
array([0.88781097, 1.74207824, 1.39951442, 0.98285695, 2.1587357 ,
       0.98285695, 1.27092473])
array([1.00000000e-15, 5.29340484e-01, 2.61225217e+00, 5.29340484e-01,
       1.62286426e+00, 7.45832107e-01, 1.16891836e+00])
array([1.00000000e-15, 2.44194997e+00, 6.99642685e-01, 6.48314520e-01,
       4.24676043e-01, 1.90532250e+00, 1.23627015e+00])
array([0.22899094, 1.80142582, 1.34016683, 1.80142582